In [1]:
import pandas as pd
import numpy as np

Load Concrete date

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check the size of the data

In [3]:
concrete_data.shape

(1030, 9)

Check columns and contents

In [4]:
concrete_data.describe

<bound method NDFrame.describe of       Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
5      266.0               114.0      0.0  228.0               0.0   
6      380.0                95.0      0.0  228.0               0.0   
7      380.0                95.0      0.0  228.0               0.0   
8      266.0               114.0      0.0  228.0               0.0   
9      475.0                 0.0      0.0  228.0               0.0   
10     198.6               132.4      0.0  192.0               0.0   
11     198.6               132.4      0.0  192.0               0.0   
12     427.5                47.5      0.0  228.0        

Let's check if we are missing values

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Predictors are all columns exceft for Strength which is the target

In [6]:
concrete_data_cols = concrete_data.columns
predictors = concrete_data[concrete_data_cols[concrete_data_cols != 'Strength']]
target = concrete_data['Strength']

Normalize the data

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
n_cols = predictors_norm.shape[1] # number of predictors to be used to build the NN

Let's get started with Keras

In [9]:
import keras

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a regression model, using 1 layer with 10 nodes, the ReLU activation function, adam as the optimizer and the mean squared error as the loss function

In [11]:
def regression_model():
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape=(n_cols,)))
    model.add(Dense(10,activation='relu',input_shape=(n_cols,)))
    model.add(Dense(10,activation='relu',input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Train and test the network

In [12]:
model = regression_model()
print(model.metrics_names)

Instructions for updating:
Colocations handled automatically by placer.
['loss']


Train the model using 50 epochs - loop 50 times reporting the mean squared errors

In [13]:
from sklearn.model_selection import train_test_split
scores = []
# fit the model
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.30, random_state=24)
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=1)
    ll = model.evaluate(X_test, y_test, verbose=0)
    scores.append(ll)
print(scores)
 

Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 12s 16ms/step - loss: 1687.7761 - val_loss: 1211.9117
Epoch 2/50
721/721 [==============================] - 1s 1ms/step - loss: 1652.3150 - val_loss: 1182.1335
Epoch 3/50
721/721 [==============================] - 2s 2ms/step - loss: 1598.0527 - val_loss: 1131.4012
Epoch 4/50
721/721 [==============================] - 1s 2ms/step - loss: 1523.9599 - val_loss: 1065.3885
Epoch 5/50
721/721 [==============================] - 1s 2ms/step - loss: 1429.3627 - val_loss: 981.6697
Epoch 6/50
721/721 [==============================] - 1s 1ms/step - loss: 1305.4216 - val_loss: 872.6707
Epoch 7/50
721/721 [==============================] - 1s 1ms/step - loss: 1139.9561 - val_loss: 734.1629
Epoch 8/50
721/721 [==============================] - 1s 1ms/step - loss: 931.5312 - val_loss: 576.1560
Epoch 9/50
721/721 [==============================] - 11s 15m

Report mean and standard deviation of fitting iterates

In [14]:
import statistics as st
print(scores)
print("Mean loss is %f" % st.mean(scores))
print("With standard deviation %f" % st.stdev(scores))

[121.65179292746733, 63.96498365926897, 39.770515324614195, 38.106199005275094, 39.2910402168348, 42.82476911575663, 42.73873381630116, 44.76691356523137, 45.231168283999544, 45.07914078351364, 44.95204705790795, 45.583309580978835, 47.16529374601, 47.867897120108495, 48.259857906106994, 47.68107256843049, 47.93465570493038, 50.96489534100282, 48.77361413653229, 48.775779292035644, 49.57574620910447, 50.36379459220615, 50.157773977730265, 50.573692667445705, 51.276125540625316, 52.07449874137212, 50.86687064556628, 51.44069060538579, 52.26359956858613, 52.278756768186504, 53.71845818183183, 53.47692992077676, 54.39993066386498, 54.96532274758546, 55.58645750866739, 55.27557785379848, 56.20694248730311, 55.37343757900991, 56.49543135065863, 56.45571442638015, 56.68653958515056, 56.72390791513387, 56.16378863189598, 57.23355722273052, 57.40134326231132, 58.09076374634184, 57.35675085864021, 57.22203300994577, 57.78895539218939, 58.31071474637029]
Mean loss is 52.743756
With standard devi

The mean for the normalized predictors with 3 NN layers is lower than Step B